# 1-step Forecasting with linear and non-linear models

In [123]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import LinearSVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from xgboost import plot_importance, plot_tree
from sklearn.model_selection import train_test_split
from sklearn import linear_model as lm
from sklearn.neighbors import KNeighborsRegressor
import sklearn.metrics as metrics

import load_data

# Plot settings
plt.rcParams['figure.figsize'] = (16, 8)
plt.rcParams['figure.dpi'] = 150
sns.set()

In [124]:
train_df, test_df, data_raw_list = load_data.load_alcohol()

combined_data = []

for i in range(len(train_df)):
    train = train_df[i]
    test = test_df[i]
    # Combine both train and test sets since the initial split was 50/50
    combined = pd.concat([train, test])
    # Sort by date
    combined['start'] = pd.to_datetime(combined['start'])
    combined = combined.sort_values(by='start')
    combined_data.append(combined)

# Dataset with all individual's data
global_data = pd.concat(combined_data, ignore_index=True)
combined_data[0].head()

,Unnamed: 0,ID,start,finish,drinks,comfortable,stressed,down,calm,pressure,...,cosT.1,sinT.1,cos2T.1,sin2T.1,cosW.1,sinW.1,dayvar.1,beepvar.1,filter.1,consec.1
0,1,1,2018-02-06 16:20:00,2/6/2018 16:22,3,7.382609,-9.817391,10.843478,-37.791304,6.173913,...,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1,4,0,1
31,2,1,2018-02-06 18:54:00,2/6/2018 18:58,0,14.382609,47.182609,7.843478,7.208696,10.173913,...,0.892979,0.450098,0.594823,0.803857,0.997777,0.066647,1,5,0,2
1,3,1,2018-02-06 20:08:00,2/6/2018 20:22,0,15.382609,12.182609,10.843478,20.208696,18.173913,...,0.418660,0.908143,-0.649448,0.760406,0.986795,0.161973,1,6,0,3
2,4,1,2018-02-06 22:29:00,2/6/2018 22:46,0,21.382609,-5.817391,-2.156522,8.208696,5.173913,...,0.108867,0.994056,-0.976296,0.216440,0.978277,0.207302,1,7,0,4
36,5,1,2018-02-07 10:52:00,2/7/2018 11:23,0,-11.617391,5.182609,0.843478,-24.791304,-4.826087,...,0.043619,-0.999048,-0.996195,-0.087156,0.777930,0.628351,2,1,0,7


## 1. Idiographic Models Regression

In [125]:
# Predict craving

# Collect train and test sets as was done in the paper
def prepare_date(idx, train_list, test_list):
    # print('Patient ID:', train_list[idx]['ID'][0])
    # print('Test Patient ID:', test_list[idx]['ID'][0])

    X_train = train_list[idx].drop(train_list[idx].columns[range(0, 61)], axis=1).fillna(0)
    y_train = train_list[idx]['craving']
    X_test = test_list[idx].drop(test_list[idx].columns[range(0, 61)], axis=1).fillna(0)
    y_test = test_list[idx]['craving']

    return X_train, y_train, X_test, y_test


def standardize(data):
    local = data.copy()
    for col in local.columns:
        local[col] = (local[col] - local[col].mean()) / np.std(local[col])
    return local


def eval_results(actual, predicted, show):
    r2 = metrics.r2_score(actual, predicted)
    rmse = metrics.mean_squared_error(actual, predicted, squared=False)
    mape = metrics.mean_absolute_percentage_error(actual, predicted)

    if show:
        print('R_squared:', r2)
        print('MAPE:', mape)
        print('RMSE:', rmse)
        print('MAE:', metrics.mean_absolute_error(actual, predicted))
        print('CORR:', np.corrcoef(predicted, actual)[0, 1])

    return r2, rmse, mape


### 1.1 Lasso Regression

In [126]:
X_train, y_train, X_test, y_test = prepare_date(1, train_df, test_df)


def lasso_reg(train_x, train_y, test_x, test_y, vis):
    X_train_loc = standardize(train_x).fillna(0)
    X_test_loc = standardize(test_x).fillna(0)
    alphas = np.arange(0.01, 20, 0.05)
    lasso = lm.LassoCV(alphas=alphas, cv=5, max_iter=100000, fit_intercept=True)
    lasso.fit(X_train_loc, train_y)
    y_predicted_test = lasso.predict(X_test_loc)

    # print('--- Lasso Regression Results ---')
    # print()
    r2, rmse, mape = eval_results(actual=test_y, predicted=y_predicted_test, show=vis)

    return r2, rmse, mape


lasso_reg(X_train, y_train, X_test, y_test, True)

R_squared: 0.2336748819426997
MAPE: 0.8602057458801604
RMSE: 21.500863785473992
MAE: 16.922922828939296
CORR: 0.5125876082907408


(0.2336748819426997, 21.500863785473992, 0.8602057458801604)

### 1.2 Elastic-Net Regression

In [127]:
def elastic_net(train_x, train_y, test_x, test_y, vis):
    X_train_loc = standardize(train_x).fillna(0)
    X_test_loc = standardize(test_x).fillna(0)

    l1_ratios = np.arange(0.01, 0.6, 0.05)
    elastic_reg = lm.ElasticNetCV(alphas=np.arange(0.01, 20, 0.05), l1_ratio=l1_ratios, cv=5, max_iter=100000,
                                  fit_intercept=True)
    elastic_reg.fit(X_train_loc, train_y)
    y_predicted_test = elastic_reg.predict(X_test_loc)

    # print('--- Elastic-Net Results ---')
    # print()
    r2, rmse, mape = eval_results(actual=test_y, predicted=y_predicted_test, show=vis)
    return r2, rmse, mape


elastic_net(X_train, y_train, X_test, y_test, True)

R_squared: 0.3105248292112299
MAPE: 0.868046768759073
RMSE: 20.394294809977133
MAE: 16.71060148405697
CORR: 0.5610027960444378


(0.3105248292112299, 20.394294809977133, 0.868046768759073)

### 1.3 Linear SVM Regression

In [128]:
def linear_svm(train_x, train_y, test_x, test_y, vis):
    X_train_loc = standardize(train_x).fillna(0)
    X_test_loc = standardize(test_x).fillna(0)

    params = [
        {'C': np.arange(0.1, 4, 0.1),
         'epsilon': np.arange(6, 7, 0.1),
         'loss': ['epsilon_insensitive'],
         'fit_intercept': [True],
         'max_iter': [10000]}]

    clf = GridSearchCV(estimator=LinearSVR(), param_grid=params, scoring='r2', cv=5)
    clf.fit(X_train_loc, train_y)
    # best_params = clf.best_params_
    # print(best_params)
    y_predicted_test = clf.predict(X_test_loc)

    # print('--- Linear-SVM Results ---')
    # print()
    r2, rmse, mape = eval_results(actual=test_y, predicted=y_predicted_test, show=vis)
    return r2, rmse, mape


linear_svm(X_train, y_train, X_test, y_test, True)

R_squared: 0.33566477371235304
MAPE: 0.8316957957366325
RMSE: 20.019029435528104
MAE: 16.0883203798706
CORR: 0.5836163446877473


(0.33566477371235304, 20.019029435528104, 0.8316957957366325)

### 1.4 K-NN Regression

In [129]:
def knn_reg(train_x, train_y, test_x, test_y, vis):
    params = [
        {'weights': ['uniform', 'distance'],
         'n_neighbors': np.arange(2, 20, 1)}]

    clf = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=params, scoring='neg_mean_squared_error', cv=2)
    clf.fit(train_x, train_y)
    # best_params = clf.best_params_
    # print(best_params)

    y_predicted_test = clf.predict(test_x)

    # print('--- kNN Regression Results ---')
    # print()
    r2, rmse, mape = eval_results(actual=test_y, predicted=y_predicted_test, show=vis)
    return r2, rmse, mape


knn_reg(X_train, y_train, X_test, y_test, True)

R_squared: 0.14247416686246706
MAPE: 1.0065771554001917
RMSE: 22.744321386225067
MAE: 17.094135426343126
CORR: 0.49368573617049394


(0.14247416686246706, 22.744321386225067, 1.0065771554001917)

### 1.5 XGBoost Regression

In [130]:
def xgboost_reg(train_x, train_y, test_x, test_y, vis):
    X_train_loc = standardize(train_x).fillna(0)
    X_test_loc = standardize(test_x).fillna(0)

    # Very simple models work better here, since there are few datapoints
    params = [
        {'objective': ['reg:squarederror'],
         'n_estimators': np.arange(1, 7, 1),
         'eval_metric': ['rmse'],
         'max_depth': np.arange(1, 5, 1)}]

    reg_xgb = GridSearchCV(xgb.XGBRegressor(), params, n_jobs=5, cv=2, scoring='neg_mean_squared_error')
    reg_xgb.fit(X_train_loc, train_y)

    y_predicted_test = reg_xgb.predict(X_test_loc)

    # print('--- XGBoost Regression Results ---')
    # print()
    r2, rmse, mape = eval_results(actual=test_y, predicted=y_predicted_test, show=vis)
    return r2, rmse, mape


xgboost_reg(X_train, y_train, X_test, y_test, True)

R_squared: 0.20080951683368675
MAPE: 0.9104222110254135
RMSE: 21.957077219941247
MAE: 17.98386691902223
CORR: 0.4737318061836795


(0.20080951683368675, 21.957077219941247, 0.9104222110254135)

### 1.6 LSTM RNN

In [131]:
import keras.layers
from keras.models import Sequential


def lstm_rnn(train_x, train_y, test_x, test_y, vis):
    X_train_loc = standardize(train_x).fillna(0)
    X_test_loc = standardize(test_x).fillna(0)
    train_x_val, train_y_val, test_x_val, test_y_val = X_train_loc.values, train_y.values, X_test_loc.values, test_y.values

    train_x_val = train_x_val.reshape((train_x_val.shape[0], 1, train_x_val.shape[1]))
    test_x_val = test_x_val.reshape((test_x_val.shape[0], 1, test_x_val.shape[1]))

    # print(train_x_val.shape)
    # print(test_x_val.shape)

    model = Sequential([
        keras.layers.LSTM(100, return_sequences=True, input_shape=(train_x_val.shape[1], train_x_val.shape[2])),
        keras.layers.Dropout(0.25),
        keras.layers.LSTM(units=50, return_sequences=True),
        keras.layers.Dropout(0.20),
        keras.layers.LSTM(units=10, return_sequences=False),
        keras.layers.Dense(units=1, activation='linear'),
    ])
    model.compile(loss='mae', optimizer='adam')
    model.fit(train_x_val, train_y_val, epochs=20, batch_size=4, verbose=0, shuffle=False)

    y_predicted_test = model.predict(test_x_val)

    r2, rmse, mape = eval_results(actual=test_y, predicted=y_predicted_test.flatten(), show=vis)
    return r2, rmse, mape


lstm_rnn(X_train, y_train, X_test, y_test, True)

R_squared: 0.09584351523480139
MAPE: 0.87308301904732
RMSE: 23.35453249726223
MAE: 20.55813424034648
CORR: 0.5087180963796669


(0.09584351523480139, 23.35453249726223, 0.87308301904732)

### 1.7 MTGNN

In [132]:
import torch
import torch.nn.functional as f
from torch_geometric_temporal.nn.recurrent.gconv_gru import GConvGRU


class RecurrentGCN(torch.nn.Module):
    def __init__(self, node_features, filters):
        super(RecurrentGCN, self).__init__()
        self.recurrent = GConvGRU(node_features, filters, 2)
        self.linear = torch.nn.Linear(filters, 1)

    def forward(self, x, edge_index, edge_weight):
        h = self.recurrent(x, edge_index, edge_weight)
        h = f.relu(h)
        h = self.linear(h)
        return h

### 1.8 Evaluating Performance on Entire Dataset

In [133]:
import warnings


def average_metrics(r2_list, rmse_list, mape_list):
    print('Average R_Squared:', np.mean(r2_list))
    print('Average RMSE:', np.mean(rmse_list))
    print('Average MAPE:', np.mean(mape_list))


def evaluate_models(train_list, test_list):
    assert len(train_list) == len(test_list)
    r2_lasso, r2_elastic, r2_svm, r2_knn, r2_xgb, r2_lstm, r2_mtgnn = ([] for _ in range(7))
    rmse_lasso, rmse_elastic, rmse_svm, rmse_knn, rmse_xgb, rmse_lstm, rmse_mtgnn = ([] for _ in range(7))
    mape_lasso, mape_elastic, mape_svm, mape_knn, mape_xgb, mape_lstm, mape_mtgnn = ([] for _ in range(7))

    patient_ids = []

    for x in range(len(train_list)):
        # Build and evaluate a model for every single patient
        train_x, train_y, test_x, test_y = prepare_date(x, train_list, test_list)
        # Elastic-Net (baseline)
        r2, rmse, mape = elastic_net(train_x, train_y, test_x, test_y,
                                     False)  # only continue with other models if this one can get a positive r2

        # Elastic-Net metrics
        patient_ids.append(train_list[x]['ID'][0])
        r2_elastic.append(max(0, r2))
        rmse_elastic.append(rmse)
        mape_elastic.append(mape)

        # Lasso Regression
        r2, rmse, mape = lasso_reg(train_x, train_y, test_x, test_y, False)
        # Lasso metrics
        r2_lasso.append(max(0, r2))
        rmse_lasso.append(rmse)
        mape_lasso.append(mape)

        # Linear-SVM
        r2, rmse, mape = linear_svm(train_x, train_y, test_x, test_y, False)
        # Linear-SVM metrics
        r2_svm.append(max(0, r2))
        rmse_svm.append(rmse)
        mape_svm.append(mape)

        # kNN Regression
        r2, rmse, mape = knn_reg(train_x, train_y, test_x, test_y, False)
        # kNN metrics
        r2_knn.append(max(0, r2))
        rmse_knn.append(rmse)
        mape_knn.append(mape)

        # XGBoost Regression
        r2, rmse, mape = xgboost_reg(train_x, train_y, test_x, test_y, False)
        # XGBoost metrics
        r2_xgb.append(max(0, r2))
        rmse_xgb.append(rmse)
        mape_xgb.append(mape)

        # LSTM RNN
        r2, rmse, mape = lstm_rnn(train_x, train_y, test_x, test_y, False)
        # LSTM metrics
        r2_lstm.append(max(0, r2))
        rmse_lstm.append(rmse)
        mape_lstm.append(mape)

    print('---- Lasso Regression Results ----')
    average_metrics(r2_lasso, rmse_lasso, mape_lasso)
    print('---------------------------------')
    print('---- Elastic-Net Results ----')
    average_metrics(r2_elastic, rmse_elastic, mape_elastic)
    print('---------------------------------')
    print('---- Linear SVM Results ----')
    average_metrics(r2_svm, rmse_svm, mape_svm)
    print('---------------------------------')
    print('---- kNN Regression Results ----')
    average_metrics(r2_knn, rmse_knn, mape_knn)
    print('---------------------------------')
    print('---- XGBoost Results ----')
    average_metrics(r2_xgb, rmse_xgb, mape_xgb)
    print('---------------------------------')
    print('---- LSTM Results ----')
    average_metrics(r2_lstm, rmse_lstm, mape_lstm)
    print('---------------------------------')

    print('Included patient list:')
    print(patient_ids)


warnings.filterwarnings("ignore")
evaluate_models(train_df, test_df)

---- Lasso Regression Results ----
Average R_Squared: 0.1770519351340509
Average RMSE: 14.717961396686716
Average MAPE: 1.5472097567998122
---------------------------------
---- Elastic-Net Results ----
Average R_Squared: 0.194006559997254
Average RMSE: 14.771363649360774
Average MAPE: 1.4156218369947697
---------------------------------
---- Linear SVM Results ----
Average R_Squared: 0.19154004883268966
Average RMSE: 14.379987420524362
Average MAPE: 1.2751523312386748
---------------------------------
---- kNN Regression Results ----
Average R_Squared: 0.11997494708105814
Average RMSE: 15.304156621838958
Average MAPE: 1.6650125231400603
---------------------------------
---- XGBoost Results ----
Average R_Squared: 0.14795224822377198
Average RMSE: 14.881679646669673
Average MAPE: 1.3614428861104988
---------------------------------
---- LSTM Results ----
Average R_Squared: 0.06194978968437973
Average RMSE: 15.585706069905084
Average MAPE: 1.1325966887492784
---------------------------

# 2. Nomothetic Models Regression

In [134]:
# Reading in the train data
train_global = pd.read_csv('data/alcohol_data/global (mean-centered)/n.train.csv')
train_global_x = train_global.drop(train_global.columns[range(0, 60)], axis=1).fillna(0)
train_global_y = train_global['craving']

train_global_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326 entries, 0 to 1325
Data columns (total 57 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   drinks.1            1326 non-null   int64  
 1   comfortable.1       1326 non-null   float64
 2   stressed.1          1326 non-null   float64
 3   down.1              1326 non-null   float64
 4   calm.1              1326 non-null   float64
 5   pressure.1          1326 non-null   float64
 6   enthusiastic.1      1326 non-null   float64
 7   happy.1             1326 non-null   float64
 8   conflict.1          1326 non-null   float64
 9   craving.1           1326 non-null   float64
 10  impulsive.1         1326 non-null   float64
 11  posexpect.1         1326 non-null   float64
 12  peerpercent.1       1326 non-null   float64
 13  wantdrink.1         1326 non-null   float64
 14  delay_grat.1        1326 non-null   float64
 15  angry.1             1326 non-null   float64
 16  drinkp

In [135]:
# Reading in the test data
test_global = pd.read_csv('data/alcohol_data/global (mean-centered)/n.test.csv')
X_test_list = []
y_test_list = []
unique_ids = np.unique(test_global['ID'])
for i in range(len(unique_ids)):
    individual = test_global.loc[test_global['ID'] == unique_ids[i]]
    X_test_list.append(individual.drop(individual.columns[range(0, 60)], axis=1).fillna(0))
    y_test_list.append(individual['craving'])

X_test_list[10].head()

,drinks.1,comfortable.1,stressed.1,down.1,calm.1,pressure.1,enthusiastic.1,happy.1,conflict.1,craving.1,...,cos2T.1,sin2T.1,cosW.1,sinW.1,dayvar.1,beepvar.1,filter.1,consec.1,NA.,NA..1
473,0,32.144068,-37.20339,-28.686441,8.09322,-2.771186,44.084746,16.728814,-9.542373,-21.728814,...,1.000000,0.000000,1.000000,0.000000,1,4,0,1,0.0,0.0
474,1,21.144068,-37.20339,-25.686441,-27.90678,-2.771186,20.084746,25.728814,-9.542373,-21.728814,...,0.484810,-0.874620,0.681086,0.732203,2,3,0,9,0.0,0.0
475,2,32.144068,-37.20339,-27.686441,34.09322,-2.771186,35.084746,35.728814,-9.542373,28.271186,...,-0.190809,0.981627,0.520371,0.853940,2,6,0,12,0.0,0.0
476,3,32.144068,-35.20339,-25.686441,45.09322,-2.771186,23.084746,35.728814,-8.542373,-21.728814,...,-0.987688,0.156435,0.443965,0.896044,2,7,0,13,0.0,0.0
477,0,32.144068,-37.20339,-26.686441,22.09322,-0.771186,-40.915254,-5.271186,56.457627,-21.728814,...,-0.920505,0.390731,0.028669,0.999589,3,1,0,16,0.0,0.0
